In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Decision Tree Regression

### Importing data

In [ ]:
df_train = pd.read_pickle(r"../input/train.pkl")
X_train = df_train.drop(["date", "count"], axis=1)
y_train = df_train["count"]
df_train.head()

In [ ]:
df_test = pd.read_pickle(r"../input/test.pkl")
X_test = df_test.drop(["date", "count"], axis=1)
y_test = df_test["count"]
df_test.head()

### Hyperparameter tuning using Grid Search

In [ ]:
parameters = {"min_samples_leaf" : list(range(1, 11))}
clf = GridSearchCV(DecisionTreeRegressor(), parameters, scoring="neg_mean_squared_error")
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
mean_squared_error(y_test, y_pred)**0.5

#### Best performing parameters
With a root-mean squared error of 53, the best performing Decision Tree Regressor is that with a minimum amount of samples per leaf of 7.

### Training regressor
Retraining the aforementioned Decision Tree Regressor with train and test data.

In [ ]:
regressor = clf.best_estimator_
regressor.fit(X_test.append(X_train), y_test.append(y_train))

### Testing regressor
The root-mean squared error of 53 mentioned in `Hyperparameter tuning using Grid Search` is more accurate, as this testing is done on data used for training.

In [ ]:
regressor.score(X_test, y_test)

In [ ]:
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)**0.5

In [ ]:
regressor

### Validating regressor
Predict the data for the dates in validation.pkl, enter data into kaggle competition. 

In [ ]:
df_validation = pd.read_pickle(r"../input/validation.pkl")
df_validation.head()

In [ ]:
X_validate = df_validation.drop(["date", "Predicted"], axis=1)

In [ ]:
y_validate = regressor.predict(X_validate)
df_validation["Predicted"] = y_validate
df_validation.head()

### Writing validation data to .csv file

In [ ]:
df_validation.rename(columns= {"date" : "id"}, inplace=True)
df_validation["id"] = df_validation["id"].dt.strftime("%Y%m%d")
df_validation[["id", "Predicted"]].to_csv("../output/val.csv", index=False)